In [1]:
import os
import sys
import imp

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

<module 'climatools.misc' from '/nuwa_cluster/home/jackyu/climatools/climatools/misc.py'>

In [2]:
def print_subroutines(fpath):
    with open(fpath, mode = 'r', encoding = 'utf-8') as file:
        code = file.read()
    subroutines = climamisc.get_subroutine_bodies_from_Fortran(code)
    for subroutine in subroutines:
        print(climamisc.get_Fortran_subroutine_name(subroutine))
        for called_subroutine in sorted(list(climamisc.get_called_Fortran_subroutine_names(subroutine))):
            print('    ', called_subroutine)
        print()

## radsw.F90

In [3]:
def getfpath_radsw():
    fpath = '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
radsw.F90'
    return fpath

In [4]:
print_subroutines(getfpath_radsw())

rad_rrtmg_sw
     CmpDayNite
     ExpDayNite
     mcica_subcol_sw
     outfld
     rrtmg_sw
     t_startf
     t_stopf

radsw_init
     get_ref_solar_band_irrad
     get_solar_band_fraction_irrad



## rrtmg_sw_rad.f90 

In [5]:
fpath_nomcica = '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
ext/rrtmg_sw/rrtmg_sw_rad.f90'

In [6]:
print_subroutines(fpath_nomcica)

rrtmg_sw
     cldprop_sw
     inatm_sw
     setcoef_sw
     spcvrt_sw

inatm_sw



### inatm_sw() no McICA
In effect this selects a column (or longitude), so the *ncol* dimension is removed, and it outputs a new cloud variable named *ice particle generalised effective size*.

INPUT:

*(ncol, nlay)*
* fraction (cldfr)
* ice water path (cicewp)
* liquid water path (cliqwp)
* ice effective radius (reice)
* water drop effective radius (reliq)

*(nbndsw, ncol, nlay)*
* optical depth (taucld)
* single scattering albedo (ssacld)
* asymmetry parameter (asmcld)

OUTPUT:

*(nlay)*
* layer fraction (cldfrac)
* ice water path (ciwp)
* liquid water path (clwp)
* liquid particle effective radius (rel)
* ice particle effective radius (rei)
* ice particle generalised effective size (dge)

*(nbndsw, nlay)*
* optical depth (non-delta scaled) (tauc)
* single scattering albedo (non-delta scaled) (ssac)
* asymmetry parameter (no-delta scaled) (asmc)

## rrtmg_sw_cldprop.f90

In [7]:
def get_fpath_for_rrtmg_sw_cldprop(mcica = True):
    if mcica:
        return '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
ext/rrtmg_mcica/rrtmg_sw_cldprmc.f90'
    else:
        return '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
ext/rrtmg_sw/rrtmg_sw_cldprop.f90'

In [8]:
print_subroutines(get_fpath_for_rrtmg_sw_cldprop(mcica = False))

cldprop_sw



### cldprop_sw()
This turns non-delta scaled optical properties to delta scaled optical properties.

INPUT:

*(nlay)*
* fraction (cldfrac)
* ice water path (ciwp)
* liquid water path (clwp)
* ice particle effective radius (rei)
* ice particle generalised effetive size (rel)
* liquid particle generalised effective size (dge)

*(nbndsw, nlay)*
* optical depth (tauc)
* single scattering albedo (ssac)
* asymmetry parameter (asmc)

OUTPUT:

*(nlay, jpband)*  
* optical depth (delta scaled) (taucloud)
* optical depth (non-delta scaled) (taucldorig)
* single scattering albedo (delta scaled) (ssacloud)
* asymmetry parameter (delta scaled) (asmcloud)

Really though, according to the code jpband here equals nbndsw, 14.

## rrtmg_sw_spcvrt.f90

In [10]:
def get_fpaths_for_spc(mcica = True):
    if mcica:
        return '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
ext/rrtmg_mcica/rrtmg_sw_spcvmc.f90'
    else:
        return '/nuwa_cluster/home/jackyu/climate_models/\
cesm1_2_2/models/atm/cam/src/physics/rrtmg/\
ext/rrtmg_sw/rrtmg_sw_spcvrt.f90'

In [11]:
print_subroutines(get_fpaths_for_spc(mcica = False))

spcvrt_sw
     reftra_sw
     taumol_sw
     vrtqdr_sw



### spcvrt_sw()

Overall structure:

    call taumol_sw()

    do spectral bands
    
        do g-points
        
            do layers
                ... <---- cloud variables appear here, they are spectral band-dependent
                
            call reftra_sw()
            
            do layers
            
            call vrtqdr_sw()
            
            do layers
            
            

INPUT:

*(nlay)*
* fraction (pclfr)

*(nlay, nbndsw)*
* optical depth (ptauc)
* asymmetry parameter (pasyc)
* single scattering albedo (pomgc)
* optical depth (non-delta scaled) (ptaucorig)

OUTPUT:
